Terra

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Mounted at /content/drive


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="yEPtugH4dS6UzzWA1pLZ")
project = rf.workspace("haha-bgzom").project("terra3-kpsuv")
version = project.version(2)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 110.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Terra3-2 in yolov8:: 100%|██████████| 3908/3908 [00:00<00:00, 6539.71it/s]


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO
import os
from PIL import Image, ImageOps
from tqdm import tqdm
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
def preprocess_images(input_folder, output_folder, size=(640, 640)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for img_name in tqdm(os.listdir(input_folder)):
        if img_name.lower().endswith(('jpg')):
            img_path = os.path.join(input_folder, img_name)
            img = Image.open(img_path)
            img = ImageOps.exif_transpose(img)
            img = img.resize(size)
            img.save(os.path.join(output_folder, img_name))

In [6]:
def copy_labels(src_folder, dst_folder):
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    for lbl in os.listdir(src_folder):
        if lbl.endswith(".txt"):
            shutil.copy(
                os.path.join(src_folder, lbl),
                os.path.join(dst_folder, lbl)
    )

In [7]:
base = "/content/Terra3-2"
splits = ["train", "valid", "test"]

for split in splits:
    img_in = os.path.join(base, split, "images")
    lbl_in = os.path.join(base, split, "labels")
    img_out = os.path.join(base, "preprocessed", split, "images")
    lbl_out = os.path.join(base, "preprocessed", split, "labels")
    print(f"Processing {split}...")
    preprocess_images(img_in, img_out, size=(640, 640))
    copy_labels(lbl_in, lbl_out)
print("Proses selesai")

Processing train...


100%|██████████| 1363/1363 [00:17<00:00, 77.72it/s]


Processing valid...


100%|██████████| 388/388 [00:04<00:00, 89.08it/s]


Processing test...


100%|██████████| 197/197 [00:02<00:00, 84.27it/s]

Proses selesai


In [8]:
model = YOLO('yolov8n.pt')

In [9]:
model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    hsv_h=0.015,
    hsv_s=0.5,
    hsv_v=0.4,
    degrees=15,
    translate=0.10,
    scale=0.3,
    shear=0.1,
    fliplr=0.5,
    flipud=0.2,
    mosaic=0.0,
    mixup=0.0,
    project='/content/drive/MyDrive/Leaf-Terra'
)

Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Terra3-2/data.yaml, degrees=15, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=1

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e96e93c80e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [10]:
model = YOLO('/content/drive/MyDrive/Leaf-Terra/train3/weights/best.pt')
metrics = model.val(data='/content/Terra3-2/data.yaml', imgsz=640)

Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 920.1±396.4 MB/s, size: 46.0 KB)
val: Scanning /content/Terra3-2/preprocessed/valid/labels.cache... 388 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 388/388 701.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 25/25 5.1it/s 4.9s
                   all        388        491      0.832      0.781       0.85      0.727
    Aphids - Kutu Daun         55         63      0.857      0.698        0.8      0.711
Cercospora - Bercak Daun         42         43        0.8      0.742       0.86      0.815
      Leaf Wilt - Layu         74         81      0.947      0.864       0.94      0.764
Phytophthora Blight - Busuk Daun         77         81       0.86      0.864      0.932      0.687
Powdery Mildew - Jamur 

In [11]:
print(metrics)

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e98671cd670>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [14]:
model.predict(source='/content/Terra3-2/preprocessed/test/images', imgsz=640, save=True, save_txt=True)


image 1/197 /content/Terra3-2/preprocessed/test/images/26_jpg.rf.04878690af5cd6a9e4c48930274e0be4.jpg: 640x640 1 Leaf Wilt - Layu, 11.7ms
image 2/197 /content/Terra3-2/preprocessed/test/images/28_jpg.rf.3e78e4852297ac31a6e973532fcbd02c.jpg: 640x640 1 Leaf Wilt - Layu, 7.2ms
image 3/197 /content/Terra3-2/preprocessed/test/images/32_jpg.rf.6cd45679f316ad29efe5ba71e1e324ca.jpg: 640x640 1 Leaf Wilt - Layu, 7.1ms
image 4/197 /content/Terra3-2/preprocessed/test/images/48_jpg.rf.d208f7eda16210c488712956b3229b35.jpg: 640x640 1 Leaf Wilt - Layu, 7.1ms
image 5/197 /content/Terra3-2/preprocessed/test/images/Aphids_original0013_jpg.rf.4d979806e19dd7a1c9cb5b3184a5dd1d.jpg: 640x640 1 Aphids - Kutu Daun, 7.1ms
image 6/197 /content/Terra3-2/preprocessed/test/images/Aphids_original0029_jpg.rf.c94a3e6626e1bf27135053ba1f7b3141.jpg: 640x640 1 Aphids - Kutu Daun, 12.2ms
image 7/197 /content/Terra3-2/preprocessed/test/images/Aphids_original0038_jpg.rf.2f23ddb473e504580c63ceeb113dd53c.jpg: 640x640 7 Aphids 

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Aphids - Kutu Daun', 1: 'Cercospora - Bercak Daun', 2: 'Leaf Wilt - Layu', 3: 'Phytophthora Blight - Busuk Daun', 4: 'Powdery Mildew - Jamur Tepung', 5: 'Sehat', 6: 'TMV - Virus'}
 obb: None
 orig_img: array([[[162, 212, 188],
         [161, 212, 185],
         [163, 211, 185],
         ...,
         [178, 206, 183],
         [171, 200, 175],
         [166, 195, 170]],
 
        [[160, 210, 186],
         [160, 211, 184],
         [161, 210, 182],
         ...,
         [198, 226, 203],
         [195, 221, 198],
         [189, 218, 193]],
 
        [[159, 210, 183],
         [158, 209, 182],
         [160, 209, 181],
         ...,
         [222, 247, 227],
         [221, 243, 224],
         [217, 243, 220]],
 
        ...,
 
        [[121, 154, 133],
         [125, 158, 137],
         [131, 164, 143],
         ...,
         [ 78,  99, 

In [15]:
model.export(format='onnx', opset=12, simplify=True)


Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/content/drive/MyDrive/Leaf-Terra/train3/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 11, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 183ms
Prepared 6 packages in 8.07s
Installed 6 packages in 242ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.19.1
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.75

requirements: AutoUpdate success ✅ 9.1s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 12...
ONNX: slimming with onnxslim 

'/content/drive/MyDrive/Leaf-Terra/train3/weights/best.onnx'